## Parsing PDB files using biopython

Experiment how to use biopython to parse a pdb file

In biopython, each structure is defined as a hierarchy of entities

- structure (each PDB file contains one structure)
- model (for X-ray structures, there is only one model, '0', for NMR there might be more)
- chain
- residue
- atom

Each entity has an attribute `id` (can also be obtained using `entity.get_id()`).
Python indexing [] can be used to access a certain child of an entity:

- `structure[0]` to retrieve first model of structure
- `model['A']` to retrieve chain 'A' of model
- `structure[0]['A']` to retrieve chain 'A' of first model of structure

There are also methods to navigate the hierarchy one level at a time:
- `entity.get_list()` returns a list of children of an entity,
- `entity.get_parent()` returns the parent of an entity

There are also methods that allow direct access of lower levels from higher levels. These are

- `entity.get_models()`
- `entity.get_chains()`
- `entity.get_residues()`
- `entity.get_atoms()`

These return an iterator, not a list. An iterator can be used directly in a for loop, or can be converted to a list using the `list(iter)` function. 


### Parsing a PDB file

Assuming we have run the notebook 'download_chothia_pdb.ipynb' which downloaded PDB files into the `PDB_DIR`, we can construct the path to the PDB file of structure '9ds1'

In [38]:
import os.path

PDB_DIR = "../data/pdbs"
pdb_id = "9ds1"
filename = os.path.join(PDB_DIR, f"{pdb_id}_chothia.pdb")

The PDB structure can then be read using biopython's `PDBParser`

In [39]:
from Bio.PDB.PDBParser import PDBParser
parser = PDBParser(PERMISSIVE=1)
structure = parser.get_structure(pdb_id, filename)

### Models

For X-ray structures, there is only one model, while for NMR structures several are possible.

We can get a list over all models using

In [40]:
models = structure.get_list() # or list(structure.get_models())
models

[<Model id=0>]

We can select the model with id 0 using

In [41]:
model0 = structure[0]
model0

<Model id=0>

In [42]:
model0.id

0

### Chains

In [43]:
chains = model0.get_list() # or list(model.get_chains())
chains

[<Chain id=A>, <Chain id=B>, <Chain id=C>, <Chain id=H>, <Chain id=L>]

In this case we are getting 5 chains. We can select chain H of the model using

In [44]:
chainH = model0['H'] # or structure[0]['H']
chainH

<Chain id=H>

In [45]:
chainH.id

'H'

### Residues

In [46]:
residues = chainH.get_list() # or list(chainH.get_residues()) 
residues
#[r.get_id() for r in residues]

[<Residue GLU het=  resseq=1 icode= >,
 <Residue VAL het=  resseq=2 icode= >,
 <Residue GLN het=  resseq=3 icode= >,
 <Residue LEU het=  resseq=4 icode= >,
 <Residue LEU het=  resseq=5 icode= >,
 <Residue GLU het=  resseq=6 icode= >,
 <Residue SER het=  resseq=7 icode= >,
 <Residue GLY het=  resseq=8 icode= >,
 <Residue GLY het=  resseq=9 icode= >,
 <Residue GLY het=  resseq=10 icode= >,
 <Residue LEU het=  resseq=11 icode= >,
 <Residue VAL het=  resseq=12 icode= >,
 <Residue GLN het=  resseq=13 icode= >,
 <Residue PRO het=  resseq=14 icode= >,
 <Residue GLY het=  resseq=15 icode= >,
 <Residue GLY het=  resseq=16 icode= >,
 <Residue SER het=  resseq=17 icode= >,
 <Residue LEU het=  resseq=18 icode= >,
 <Residue ARG het=  resseq=19 icode= >,
 <Residue LEU het=  resseq=20 icode= >,
 <Residue SER het=  resseq=21 icode= >,
 <Residue CYS het=  resseq=22 icode= >,
 <Residue ALA het=  resseq=23 icode= >,
 <Residue VAL het=  resseq=24 icode= >,
 <Residue SER het=  resseq=25 icode= >,
 <Residue

Let's look at the `id` of a residue

In [47]:
residues[23].id

(' ', 24, ' ')

The residue id is a tuple consisting of three elements, `(het_flag, residue_number, insertion_code)`.

If the residue is a proper amino acid, the `het_flag` is ' ' (space, not empty!).

If the residue is a hetero atom (ligand, water, other molecule) then the 
`het_flag` is not ' ', e.g. 'W' for water, 'H_GLC' for glucose, etc.

The `insertion_code` is needed when there is an insertion within a CDR region (Chothia numbering).
Otherwise it is a space ' ' as well.

To access these elements, we can destructure the `id` of a residue

In [48]:
het_flag, residue_number, insertion_code = residues[23].id

or we can index into the id tuple, e.g.

In [49]:
het_flag = residues[23].id[0]

We can obtain the name of the residue using the `get_resname()` method.

Examples:

Residue 53 (python is 0-based, so index 52 refers to element 53) is GLY. 

In [50]:
residues[52].get_resname()

'GLY'

But as it seems to be an insertion within a CDR region, the insertion_code is 'A'.

In [51]:
residues[52].id

(' ', 52, 'A')

Residue 233  is a water molecule. This means the first entry of the id is 'W'.

In [52]:
residues[232].get_resname()

'HOH'

In [53]:
residues[232].id

('W', 222, ' ')

### Atoms

As the last level of the hierarchy, we can retrieve the atoms of a residue.

In [54]:
atoms = residues[52].get_list()  # or list(residues[52].get_atoms())
atoms

[<Atom N>, <Atom CA>, <Atom C>, <Atom O>]

In [55]:
atoms[2].id

'C'

All entities below structure also have an attribute `full_id` that shows the placing of the entity in the hierarchy.

In [56]:
atoms[2].full_id

('9ds1', 0, 'H', (' ', 52, 'A'), ('C', ' '))

To get the coordinates of an atom, we can use its `coord` property. It returns the x, y, z coordinates as a numpy array.

In [57]:
atoms[2].coord

array([ 16.06 , -62.04 ,  26.474], dtype=float32)

Up to now we have walked the hierarchyy downwards using the `get_list()` method to retrieve children of an entity. We went from structure to model to chain to residue to atom. We can also walk the hierarchy upwards using the `get_parent()` method. I.e. given an atom, we can find the residue it belongs to.

In [58]:
atoms[3].get_parent()

<Residue GLY het=  resseq=52 icode=A>

It is also possible to jump over levels in the hierarchy, using methods `get_models()`, `get_chains()`, `get_residues()`, `get_atoms()`. Note that these methods return iterators, not lists. To convert them to lists, use the `list` function.

To retrieve all chains of a structure, irrespective to which model they belong

In [59]:
[c.id for c in structure.get_chains()]

['A', 'B', 'C', 'H', 'L']

To retrieve all atoms of chain H

In [60]:
atoms = structure[0]['H'].get_atoms()
len(list(atoms))

1708

### Exercise

Make sure you have downloaded the PDB files as in 'download_chothia_pdb.ipynb' notebook.

Consider pdb_id '9ds2'

- create the path to the file
- parse the file
- retrieve all residues of the 'H' chain (this can be done in a single line in python)
- filter on amino acid residues (i.e. the `het_flag` of the parent residue is ' ', space not empty).
  Here one can use a list comprehension in python:
  `[res for res in residues if res.id[0] == ' ']`



In [66]:
pdb_id = '9ds2'
filename = os.path.join(PDB_DIR, f"{pdb_id}_chothia.pdb")

parser = PDBParser(PERMISSIVE=1)
structure = parser.get_structure(pdb_id, filename)

residues = structure[0]['H'].get_residues()

aa_residues = [res for res in residues if res.id[0] == ' ']

The `get_resname()` method of the residues returns a 3-letter code like 'ALA' or 'TRP'. 
We need to create a mapping from 3-letter codes to one-letter codes 'A' or 'W'. 

- create a python dictionary `three2one` that maps from three-letter protein codes to one-letter codes (ask Chat-GPT)

- create a list of one-letter codes from the amino acid residues above. Use a list comprehension. This can be done using a list comprehension. 

- create a string from this list. Use `"".join(reslist)`



In [70]:
three2one = {
    'ALA': 'A',
    'ARG': 'R',
    'ASN': 'N',
    'ASP': 'D',
    'CYS': 'C',
    'GLN': 'Q',
    'GLU': 'E',
    'GLY': 'G',
    'HIS': 'H',
    'ILE': 'I',
    'LEU': 'L',
    'LYS': 'K',
    'MET': 'M',
    'PHE': 'F',
    'PRO': 'P',
    'SER': 'S',
    'THR': 'T',
    'TRP': 'W',
    'TYR': 'Y',
    'VAL': 'V',
    'SEC': 'U',  # Selenocysteine
    'PYL': 'O',  # Pyrrolysine
    'ASX': 'B',  # Aspartic acid or Asparagine
    'GLX': 'Z',  # Glutamic acid or Glutamine
    'XLE': 'J',  # Leucine or Isoleucine
    'UNK': 'X'   # Unknown
}

r = [three2one[res.get_resname()] for res in aa_residues]

''.join(r)

'QVQLVESGGGVVQPGKSLRLSCAASGFTFSAYGIHWVRQAPGKGLEWVAFISFDGGSKYYADSVKGRFTISRDNSKNTLYVHMNSLRVEDTAVYYCARDRGGKYPPSMGFFDPWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKRVEPKSC'

Write a function `extract_aa_sequence(pdb_id, chain, limit)` that

- creates the path to the PDB file for pdb_id
- parses the file
- retrieves all residues of the given chain (e.g. 'H')
- filters on `het_flag == ' '` and `res_number <= limit`.
- returns a list of one-letter protein codes


In [71]:
def extract_aa_sequence(pdb_id, chain, limit):
    filename = os.path.join(PDB_DIR, f"{pdb_id}_chothia.pdb")

    parser = PDBParser(PERMISSIVE=1)
    structure = parser.get_structure(pdb_id, filename)

    residues = structure[0][chain].get_residues()

    aa_residues = [res for res in residues if res.id[0] == ' ' and res.id[1] <= limit]

    return ''.join([three2one[res.get_resname()] for res in aa_residues])

In [72]:
extract_aa_sequence('9ds2', 'H', 10)

'QVQLVESGGG'